In [5]:
import cv2

def confronta_video(video1, video2):
    cap1 = cv2.VideoCapture(video1)
    cap2 = cv2.VideoCapture(video2)

    # Verifica se i video possono essere aperti correttamente
    if not cap1.isOpened():
        print(f"Impossibile aprire il video 1: {video1}")
        return

    if not cap2.isOpened():
        print(f"Impossibile aprire il video 2: {video2}")
        return

    # Ottieni il numero totale di frame nei due video
    total_frames1 = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frames2 = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))

    #print(f"Numero totale di frame nel video 1: {total_frames1}")
    #print(f"Numero totale di frame nel video 2: {total_frames2}")

    # Imposta il numero massimo di frame da confrontare
    max_frames = min(total_frames1, total_frames2)

    # Inizializza il conteggio dei frame uguali
    frame_uguali = 0

    for frame_index in range(max_frames):
        # Leggi i frame dai due video
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        # Se uno dei due frame non può essere letto, esci dal ciclo
        if not ret1 or not ret2:
            break
        
        # Ridimensiona i frame per adattarli alle stesse dimensioni
        frame1 = cv2.resize(frame1, (640, 480))
        frame2 = cv2.resize(frame2, (640, 480))

        # Converti i frame in scala di grigi per facilitare il confronto
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calcola la differenza assoluta tra i due frame
        diff = cv2.absdiff(gray1, gray2)

        # Se la differenza è inferiore a una certa soglia, consideriamo i frame uguali
        if cv2.countNonZero(diff) == 0:
            frame_uguali += 1

    # Calcola la percentuale di somiglianza tra i due video
    percentuale_somiglianza = (frame_uguali / max_frames) * 100

    # Chiudi i video
    cap1.release()
    cap2.release()

    return percentuale_somiglianza

confronta_video("1.mp4", "0.mp4") 

Impossibile aprire il video 1: 1.mp4


OpenCV: Couldn't read video stream from file "1.mp4"
OpenCV: Couldn't read video stream from file "0.mp4"


In [18]:
import os
import concurrent.futures
from tqdm import tqdm

def confronta_video_cartelle_parallel(cartella1, cartella2, soglia_somiglianza):
    # Recupera la lista dei file video nella prima cartella
    video1_list = [file for file in os.listdir(cartella1) if file.endswith(('.mp4', '.avi'))]

    # Recupera la lista dei file video nella seconda cartella
    video2_list = [file for file in os.listdir(cartella2) if file.endswith(('.mp4', '.avi'))]

    # Creeo una matrice dei risultati, righe = video1, colonne = video2 usando un dict
    results = {video1: {video2: None for video2 in video2_list} for video1 in video1_list}

    # Crea una lista di coppie di video da confrontare
    video_pairs = [(video1, video2) for video1 in video1_list for video2 in video2_list]

    # Funzione per confrontare un singolo paio di video
    def confronta_video_pair(video_pair):
        print(video_pair)
        video1 = video_pair[0]
        video2 = video_pair[1]

        # Crea i percorsi completi per i video
        video1_path = os.path.join(cartella1, video1)
        video2_path = os.path.join(cartella2, video2)

        # Esegui il confronto tra i video
        percentuale_somiglianza = confronta_video(video1_path, video2_path)

        return percentuale_somiglianza

    # Esegui il confronto per ogni coppia di video
    for pair in tqdm(video_pairs):
        results[pair[0]][pair[1]] = confronta_video_pair(pair)

    # Restitusici la matrice dei risultati
    return results

# Utilizzo della funzione
cartella1 = "../data/EXAMPLE/0"
cartella2 = "../data/EXAMPLE/0"
soglia_somiglianza = 30

res = confronta_video_cartelle_parallel(cartella1, cartella2, soglia_somiglianza)

for item in res:
    print(item, res[item])


  0%|          | 0/9 [00:00<?, ?it/s]

('Video0.mp4', 'Video0.mp4')


 11%|█         | 1/9 [00:05<00:46,  5.76s/it]

('Video0.mp4', 'Video1.mp4')


 22%|██▏       | 2/9 [00:08<00:28,  4.05s/it]

...

('Video2.mp4', 'Video1.mp4')


 89%|████████▉ | 8/9 [00:22<00:02,  2.24s/it]

('Video2.mp4', 'Video2.mp4')


100%|██████████| 9/9 [00:22<00:00,  2.54s/it]

Video0.mp4 {'Video0.mp4': 100.0, 'Video1.mp4': 0.0, 'Video2.mp4': 0.0}
Video1.mp4 {'Video0.mp4': 0.0, 'Video1.mp4': 100.0, 'Video2.mp4': 0.0}
Video2.mp4 {'Video0.mp4': 0.0, 'Video1.mp4': 0.0, 'Video2.mp4': 100.0}
